In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
from pathlib import Path

In [ ]:
TREATMENT_COLOUR = {
    "Control": "#cb9e59",       # India Yellow
    "50 μl":   "#5d82a1",       # Ultra Marine Blue
    "500 μl":  "#a53c49",       # Rectory Red
}


def extract_organism(label: str) -> str:
    """Return plain‑text organism name from the LaTeX label."""
    m = re.match(r"\$\s*\\it\{([^}]*)\}", label)
    if m:
        return m.group(1).strip()
    return "Control" if "Control" in label else "Unknown"


def assign_treatment(label: str) -> str:
    if label == "Control":
        return "Control"
    if "50 μl" in label:
        return "50 μl"
    if "500 μl" in label:
        return "500 μl"
    return ""


def make_plot(csv_path: Path, output_png: Path = None) -> None:
    df = pd.read_csv(csv_path)

    # Ensure numeric columns
    df["Average grey/plant (a.u. 16 bit)"] = pd.to_numeric(
        df["Average grey/plant (a.u. 16 bit)"], errors="coerce"
    )
    df["Time"] = pd.to_numeric(df["Time"], errors="coerce")

    # Day index (1‑based)
    df["Day"] = df["Time"] + 1

    # Derived columns
    df["Organism"] = df["Label"].apply(extract_organism)
    df["Treatment"] = df["Label"].apply(assign_treatment)

    organisms = ["E. coli", "A. tumefaciens", "P. syringae"]
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
    fig.suptitle(
        "Treatment of jasmonic‑acid‑sensitive $\\it{A. thaliana}$ with bacteria",
        fontsize=18,
        y=0.98,
    )

    for ax, org in zip(axes, organisms):
        subset = df[(df["Organism"] == org) | (df["Label"] == "Control")]

        # Mean per (day, treatment)
        means = (
            subset.groupby(["Day", "Treatment"])["Average grey/plant (a.u. 16 bit)"]
            .mean()
            .reset_index()
        )

        for treat in ["Control", "50 μl", "500 μl"]:
            data_line = means[means["Treatment"] == treat]
            ax.plot(
                data_line["Day"],
                data_line["Average grey/plant (a.u. 16 bit)"],
                marker="o",
                label=treat,
                color=TREATMENT_COLOUR[treat],
                linewidth=2.2,
                markersize=5,
            )

        ax.set_xlim(1, 7)
        ax.set_xticks(range(1, 8))
        ax.set_xlabel("Time post‑treatment (days)", fontsize=12)
        ax.set_title(org, fontsize=14)
        ax.tick_params(labelsize=10)
        ax.grid(alpha=0.3)
        ax.legend(title="Treatment", frameon=False, fontsize=9, title_fontsize=10, loc="upper left")

    axes[0].set_ylabel("Luminescence (RLU)", fontsize=12)

    plt.tight_layout()

    if output_png:
        plt.savefig(output_png, dpi=300, bbox_inches="tight")
        print(f"Plot saved to {output_png}")
    else:
        plt.show()

In [ ]:
csv_arg = '' # .Insert here path to data
out_png = '' # Insert here path to save figure
make_plot(csv_arg, out_png)